## Функция ранжирования bm25

Для обратного индекса есть общепринятая формула для ранжирования *Okapi best match 25* ([Okapi BM25](https://ru.wikipedia.org/wiki/Okapi_BM25)).    
Пусть дан запрос $Q$, содержащий слова  $q_1, ... , q_n$, тогда функция BM25 даёт следующую оценку релевантности документа $D$ запросу $Q$:

$$ score(D, Q) = \sum_{i}^{n} \text{IDF}(q_i)*\frac{TF(q_i,D)*(k+1)}{TF(q_i,D)+k(1-b+b\frac{l(d)}{avgdl})} $$ 
где   
>$TF(q_i,D)$ - частота слова $q_i$ в документе $D$      
$l(d)$ - длина документа (количество слов в нём)   
*avgdl* — средняя длина документа в коллекции    
$k$ и $b$ — свободные коэффициенты, обычно их выбирают как $k$=2.0 и $b$=0.75   
$$$$
$\text{IDF}(q_i)$ - это модернизированная версия IDF: 
$$\text{IDF}(q_i) = \log\frac{N-n(q_i)+0.5}{n(q_i)+0.5},$$
>> где $N$ - общее количество документов в коллекции   
$n(q_i)$ — количество документов, содержащих $q_i$

### __Задача 1__:    
Напишите два поисковика на *BM25*. Один через подсчет метрики по формуле для каждой пары слово-документ, второй через умножение матрицы на вектор. 

Сравните время работы поиска на 100к запросах. В качестве корпуса возьмем 
[Quora question pairs](https://www.kaggle.com/loopdigga/quora-question-pairs-russian).

In [1]:
import os

In [2]:
import pandas as pd

In [3]:
data = pd.read_csv('C:/Users/kapru/Desktop/infosearch-master/quora_question_pairs_rus.csv')

In [4]:
data.head()

,Unnamed: 0,question1,question2,is_duplicate
0,0,Какова история кохинор кох-и-ноор-бриллиант,"что произойдет, если правительство Индии украд...",0
1,1,как я могу увеличить скорость моего интернет-с...,как повысить скорость интернета путем взлома ч...,0
2,2,"почему я мысленно очень одинок, как я могу это...","найти остаток, когда математика 23 ^ 24 матема...",0
3,3,которые растворяют в воде быстро сахарную соль...,какая рыба выживет в соленой воде,0
4,4,астрология: я - луна-колпачок из козерога и кр...,Я тройная луна-козерог и восхождение в козерог...,1


In [5]:
pip install pymorphy2

Note: you may need to restart the kernel to use updated packages.


**Способ 1:** подсчет метрики по формуле для каждой пары слово-документ

In [6]:
import pymorphy2
morph = pymorphy2.MorphAnalyzer()

In [7]:
inputs = []
for row in data['question1'][0:10]:
    inputs.append(str(row))

In [8]:
texts = []
for row in data['question2']:
    texts.append(str(row))

In [9]:
import re
import nltk
nltk.download('stopwords')
stemmer = nltk.stem.PorterStemmer()
stopwords = set(nltk.corpus.stopwords.words('russian'))

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\kapru\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [10]:
#tokenize documents
BEGIN = 10000
N = 10000

allWordsAllTexts = []
for text in texts[BEGIN:BEGIN+N]:
    allWordsInOneText = []
    text = re.sub('[^а-яё ]', '', text.lower())
    for word in text.split():
        if word not in stopwords:
            p = morph.parse(word)[0]
            if p not in allWordsInOneText:
                allWordsInOneText.append(p.normal_form)
    allWordsAllTexts.append(allWordsInOneText)

In [11]:
allWordsAllTexts[5]

['получить', 'доступ', 'анонимный', 'вопрос']

In [12]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer()

In [13]:
arrayString = []
for text in allWordsAllTexts:
    s = ' '.join(text)
    arrayString.append(s)

In [14]:
arrayString[5]

'получить доступ анонимный вопрос'

In [15]:
X = vectorizer.fit_transform(arrayString)

In [16]:
Matrix = X.toarray()

In [17]:
lens = []
for i in arrayString:
    lens.append(len(i))

In [18]:
avgdl = sum([len(doc) for doc in allWordsAllTexts])/len(allWordsAllTexts)
avgdl

5.7591

In [19]:
import numpy as np

In [20]:
tf_matrix = Matrix/np.array(lens).reshape((-1,1))

C:\ProgramFiles\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: RuntimeWarning: invalid value encountered in true_divide
  """Entry point for launching an IPython kernel.


In [21]:
tf_matrix

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [22]:
from math import log
words = vectorizer.get_feature_names()
num_doc_qi = np.count_nonzero(Matrix, axis=0)

In [23]:
num_doc_qi

array([1, 3, 1, ..., 2, 7, 1], dtype=int64)

In [24]:
def idf(word):
    word_id = words.index(word)
    num = num_doc_qi[word_id]
    score = log((N - num+0.5)/(num+0.5))
    return score

In [25]:
idf('звёздный')

7.33798804376911

In [26]:
all_idfs = []
for word in words:
    score = idf(word)
    all_idfs.append(score)

In [27]:
def tokenize(text):
    allWordsInOneInput = []
    text = re.sub('[^а-яё ]', '', text.lower())
    for word in text.split():
        p = morph.parse(word)[0]
        allWordsInOneInput.append(p.normal_form)
    return allWordsInOneInput

In [28]:
tokenize('звёздное небо')

['звёздный', 'небо']

In [29]:
def bm25_for_words(query, b):
    k = 2
    q_tokenized = tokenize(query)
    results = []
    for i in range(N):
        bm25 = 0
        for j in range(len(q_tokenized)):
            word = q_tokenized[j]
            if word in words:
                word_id = words.index(word)
                l = lens[i]
                tf_val = tf_matrix[(i,word_id)]
                if tf_val != 0:
                    tf_new = (tf_val*(k+1.0))/tf_val+k*(1.0-b+b*(l/avgdl))
                    bm25 += all_idfs[word_id]*tf_new
        results.append(bm25)
    return results

In [30]:
bm = bm25_for_words('джедай', 0.75)

In [31]:
bm_id = bm.index(max(bm))

In [32]:
print('Лучшее совпадение - ' + '"' + texts[BEGIN+bm_id] + '"')

Лучшее совпадение - "почему голливудский учет, где налог избегается, лживая о прибыли - например, возвращение джедаев, по-видимому, никогда не получало прибыли, которую можно было продолжить"


In [33]:
import time
start_time = time.time()
for sent in inputs:
    bm = bm25_for_words(sent, 0.75)
    bm_id = bm.index(max(bm))
    final = texts[BEGIN+bm_id]
print("--- %s seconds ---" % (time.time() - start_time))

--- 106.97230410575867 seconds ---


**Способ:** 2 без умножения векторов на матрицу

In [34]:
documents = {}
for idx, text in enumerate(allWordsAllTexts):
    documents[idx] = text

In [35]:
documents[9621]

['йодский', 'звёздный', 'война', 'стать', 'джедай']

In [36]:
from collections import defaultdict
    
inverted_index = defaultdict(set)

for docid, terms in documents.items():
    for term in terms:
        inverted_index[term].add(docid) 

In [37]:
inverted_index['звёздный']

{768, 2402, 2788, 4641, 6861, 9621}

In [38]:
import math
from math import log

In [39]:
#Building a TF-IDF representation using BM25 

NO_DOCS = len(documents) #Number of documents

AVG_LEN_DOC = sum([len(doc) for doc in documents.values()])/len(documents) #Average length of documents

#The function below takes the documentid, and the term, to calculate scores for the tf and idf
#components, and multiplies them together.
def tf_idf_score(k1,b,term,docid):  
    
    ft = len(inverted_index[term]) 
    term = stemmer.stem(term.lower())
    fdt =  documents[docid].count(term)
    
    idf_comp = math.log((NO_DOCS - ft + 0.5)/(ft+0.5))
    
    tf_comp = ((k1 + 1)*fdt)/(k1*((1-b) + b*(len(documents[docid])/AVG_LEN_DOC))+fdt)
    
    return idf_comp * tf_comp

#Function to create tf_idf matrix without the query component
def create_tf_idf(k1,b):
    tf_idf = defaultdict(dict)
    for term in set(inverted_index.keys()):
        for docid in inverted_index[term]:
            tf_idf[term][docid] = tf_idf_score(k1,b,term,docid)
    return tf_idf

In [40]:
NO_DOCS

10000

In [41]:
AVG_LEN_DOC

5.7591

In [42]:
tf_idf = create_tf_idf(2.0,0.75)

In [43]:
#Function to retrieve query component
def get_qtf_comp(k3,term,fqt):
    return ((k3+1)*fqt[term])/(k3 + fqt[term])


#Function to retrieve documents || Returns a set of documents and their relevance scores. 
def retr_docs(query,result_count):
    q_terms = [stemmer.stem(term.lower()) for term in str(query).split() if term not in stopwords] #Removing stopwords from queries
    fqt = {}
    for term in q_terms:
        fqt[term] = fqt.get(term,0) + 1
    
    scores = {}
    
    for word in fqt.keys():
        #print word + ': '+ str(inverted_index[word])
        for document in inverted_index[word]:
            scores[document] = scores.get(document,0) + (tf_idf[word][document]*get_qtf_comp(0,word,fqt)) #k3 chosen as 0 (default)
    
    return sorted(scores.items(),key = lambda x : x[1] , reverse=True)[:result_count] 

In [44]:
retr_docs("путешествие",5)

[(719, 9.115033780888819),
 (71, 8.074397895821996),
 (3669, 8.074397895821996),
 (7163, 8.074397895821996),
 (7317, 7.247026734974991)]

In [45]:
documents[7317]

['почему', 'путешествие', 'время', 'парадокс']

In [46]:
import time
start_time = time.time()
for row in inputs:
    retr_docs(row,1)
print("--- %s seconds ---" % (time.time() - start_time))

--- 0.00797891616821289 seconds ---


**Вывод:** второй способ быстрее

### __Задача 2__:    



Выведите 10 первых результатов и их близость по метрике BM25 по запросу **рождественские каникулы** на нашем корпусе  Quora question pairs. 

In [47]:
retr_docs("рождественские каникулы",10)

[(7709, 8.879069318533192), (4036, 2.598833886167791)]

In [48]:
documents[7709]

['мочь', 'провести', 'летний', 'каникулы', 'осмысленно']

In [49]:
documents[4036]

['делать',
 'бакалавр',
 'область',
 'компьютерный',
 'наука',
 'сделать',
 'мой',
 'метр',
 'семестр',
 'несколько',
 'день',
 'месяц',
 'летний',
 'каникулы',
 'вперёд',
 'хороший',
 'способ',
 'мочь',
 'использовать',
 'месяц',
 'мочь',
 'приземлиться',
 'хороший',
 'стажировка',
 'будущее',
 'вершина',
 'компания',
 'надпись',
 'тип',
 'том',
 'далее']

### __Задача 3__:    

Посчитайте точность поиска при 
1. BM25, b=0.75 
2. BM15, b=0 
3. BM11, b=1

precision = (relevant documents & retrieved documents)/retrieved documents

In [50]:
bm25 = bm25_for_words('кофе', 0.75)
for i, freq in enumerate(bm25):
    bm25[i] = (BEGIN+i, freq)
bm25 = list(filter(lambda x: x[1] > 0, bm25))
bm25.sort(key = lambda x: x[1], reverse = True)

In [51]:
for res in bm25:
    print(res[1], texts[res[0]])

186.34018965405718 пить кофе на голодный желудок оказывает слабительное воздействие на меня, это то же самое для других людей
165.72690149939353 сколько ключевых слов есть на языке программирования сценариев кофе в последней версии
163.85296621260593 глава моей компании с радостью согласился сесть и взять со мной кофе, что я должен спросить у него
120.75245461649102 какие наиболее частые неисправности кофе и как их обнаружить
85.14768416752652 как я могу получить большую чашку кофе в Starbucks
77.6519430203761 каковы преимущества никогда не пить кофе
60.78652543928766 варится кофе плохо


In [52]:
# из семи документов два по факту относятся к другому контексту (не напрямую связанном с кофе)
precision_bm25=2/7
precision_bm25

0.2857142857142857

In [53]:
bm15 = bm25_for_words('кофе', 0)
for i, freq in enumerate(bm15):
    bm15[i] = (BEGIN+i, freq)
bm15 = list(filter(lambda x: x[1] > 0, bm15))
bm15.sort(key = lambda x: x[1], reverse = True)

In [54]:
for res in bm15:
    print(res[1], texts[res[0]])

35.97393570046166 сколько ключевых слов есть на языке программирования сценариев кофе в последней версии
35.97393570046166 каковы преимущества никогда не пить кофе
35.97393570046166 как я могу получить большую чашку кофе в Starbucks
35.97393570046166 глава моей компании с радостью согласился сесть и взять со мной кофе, что я должен спросить у него
35.97393570046166 пить кофе на голодный желудок оказывает слабительное воздействие на меня, это то же самое для других людей
35.97393570046166 какие наиболее частые неисправности кофе и как их обнаружить
35.97393570046166 варится кофе плохо


In [55]:
bm11 = bm25_for_words('кофе', 1)
for i, freq in enumerate(bm11):
    bm11[i] = (BEGIN+i, freq)
bm11 = list(filter(lambda x: x[1] > 0, bm11))
bm11.sort(key = lambda x: x[1], reverse = True)

In [56]:
for res in bm11:
    print(res[1], texts[res[0]])

236.46227430525573 пить кофе на голодный желудок оказывает слабительное воздействие на меня, это то же самое для других людей
208.9778900990375 сколько ключевых слов есть на языке программирования сценариев кофе в последней версии
206.47930971665403 глава моей компании с радостью согласился сесть и взять со мной кофе, что я должен спросить у него
149.01196092183415 какие наиболее частые неисправности кофе и как их обнаружить
101.53893365654814 как я могу получить большую чашку кофе в Starbucks
91.54461212701425 каковы преимущества никогда не пить кофе
69.05738868556298 варится кофе плохо
